In [13]:
import numpy as np
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize

def ppmi_weighting(docs):
   
    vectorizer = CountVectorizer(tokenizer=lambda text: text.split(), binary=True)
    X = vectorizer.fit_transform(docs)
    co_occurrence_matrix = (X.T * X)

   
    word_counts = np.array(X.sum(axis=0)).squeeze()
    total_word_count = word_counts.sum()

    
    ppmi_matrix = np.zeros(co_occurrence_matrix.shape)

   
    num_docs = len(docs)
    for i, (word1, idx1) in enumerate(vectorizer.vocabulary_.items()):
        for j, (word2, idx2) in enumerate(vectorizer.vocabulary_.items()):
            co_occurrences = co_occurrence_matrix[idx1, idx2]
            if co_occurrences == 0:
                ppmi_matrix[idx1, idx2] = 0
            else:
                word1_count = word_counts[idx1]
                word2_count = word_counts[idx2]
                pmi = np.log((co_occurrences * num_docs) / (word1_count * word2_count))
                ppmi = max(0, pmi)
                ppmi_matrix[idx1, idx2] = ppmi

   
    ppmi_matrix = normalize(ppmi_matrix, norm='l2', axis=1)

    return ppmi_matrix

# Example usage:
docs = [
    "This is a sample document containing some words.",
    "Another document with different words but containing some similar words."
]

ppmi_matrix = ppmi_weighting(docs)
print(ppmi_matrix)


[[0.5        0.         0.         0.         0.         0.
  0.5        0.5        0.         0.         0.5        0.
  0.         0.        ]
 [0.         0.40824829 0.40824829 0.         0.40824829 0.
  0.         0.         0.40824829 0.         0.         0.40824829
  0.40824829 0.        ]
 [0.         0.40824829 0.40824829 0.         0.40824829 0.
  0.         0.         0.40824829 0.         0.         0.40824829
  0.40824829 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.40824829 0.40824829 0.         0.40824829 0.
  0.         0.         0.40824829 0.         0.         0.40824829
  0.40824829 0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.5        0.         0.         0.         0.         0.
  0.5        0.5        0.         0.         

In [14]:
def ppmi_weighting(docs, window_size=5):
    
    tokenized_docs = [doc.split() for doc in docs]
    co_occurrence_matrix = defaultdict(lambda: defaultdict(int))

    for doc in tokenized_docs:
        for i, word in enumerate(doc):
            start = max(0, i - window_size)
            end = min(len(doc), i + window_size + 1)
            context = doc[start:end]
            for j in range(len(context)):
                if context[j] != word:
                    co_occurrence_matrix[word][context[j]] += 1

   
    word_counts = defaultdict(int)
    for doc in tokenized_docs:
        for word in doc:
            word_counts[word] += 1
    total_word_count = sum(word_counts.values())

  
    ppmi_matrix = defaultdict(lambda: defaultdict(float))

   
    num_docs = len(docs)
    for word, context in co_occurrence_matrix.items():
        for co_word, co_count in context.items():
            if co_count == 0:
                ppmi_matrix[word][co_word] = 0
            else:
                pmi = np.log((co_count * num_docs) / (word_counts[word] * word_counts[co_word]))
                ppmi = max(0, pmi)
                ppmi_matrix[word][co_word] = ppmi

    
    vocab = sorted(word_counts.keys())
    ppmi_array = np.zeros((len(vocab), len(vocab)))
    for i, word in enumerate(vocab):
        for j, co_word in enumerate(vocab):
            ppmi_array[i][j] = ppmi_matrix[word][co_word]

   
    ppmi_array = normalize(ppmi_array, norm='l2', axis=1)

    return ppmi_array, vocab

# Example usage:
docs = [
    "This is a sample document containing some words.",
    "Another document with different words but containing some similar words."
]

ppmi_matrix, vocab = ppmi_weighting(docs, window_size=5)
print("Vocabulary:", vocab)
print("PPMI Matrix:")
print(ppmi_matrix)

Vocabulary: ['Another', 'This', 'a', 'but', 'containing', 'different', 'document', 'is', 'sample', 'similar', 'some', 'with', 'words', 'words.']
PPMI Matrix:
[[0.         0.         0.         0.5        0.         0.5
  0.         0.         0.         0.         0.         0.5
  0.5        0.        ]
 [0.         0.         0.57735027 0.         0.         0.
  0.         0.57735027 0.57735027 0.         0.         0.
  0.         0.        ]
 [0.         0.57735027 0.         0.         0.         0.
  0.         0.57735027 0.57735027 0.         0.         0.
  0.         0.        ]
 [0.4472136  0.         0.         0.         0.         0.4472136
  0.         0.         0.         0.4472136  0.         0.4472136
  0.4472136  0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.4472136  0.         0.         0.4472136  0.         0.
  0.         0.         0.         0.4472

In [15]:
def ppmi_weighting(docs, window_size=5):
    
    tokenized_docs = [doc.split() for doc in docs]
    co_occurrence_matrix = defaultdict(lambda: defaultdict(int))

    for doc in tokenized_docs:
        for i, word in enumerate(doc):
            start = max(0, i - window_size)
            end = min(len(doc), i + window_size + 1)
            context = doc[start:end]
            for j in range(len(context)):
                if context[j] != word:
                    co_occurrence_matrix[word][context[j]] += 1

   
    word_counts = defaultdict(int)
    for doc in tokenized_docs:
        for word in doc:
            word_counts[word] += 1
    total_word_count = sum(word_counts.values())

    
    for word, count in word_counts.items():
        word_counts[word] = 1 + np.log(count)  
   
    ppmi_matrix = defaultdict(lambda: defaultdict(float))

   
    num_docs = len(docs)
    for word, context in co_occurrence_matrix.items():
        for co_word, co_count in context.items():
            if co_count == 0:
                ppmi_matrix[word][co_word] = 0
            else:
                pmi = np.log((co_count * num_docs) / (word_counts[word] * word_counts[co_word]))
                ppmi = max(0, pmi)
                ppmi_matrix[word][co_word] = ppmi

   
    vocab = sorted(word_counts.keys())
    ppmi_array = np.zeros((len(vocab), len(vocab)))
    for i, word in enumerate(vocab):
        for j, co_word in enumerate(vocab):
            ppmi_array[i][j] = ppmi_matrix[word][co_word]

   
    ppmi_array = normalize(ppmi_array, norm='l2', axis=1)

    return ppmi_array, vocab

# Example usage:
docs = [
    "This is a sample document containing some words.",
    "Another document with different words but containing some similar words."
]

ppmi_matrix, vocab = ppmi_weighting(docs, window_size=5)
print("Vocabulary:", vocab)
print("PPMI Matrix:")
print(ppmi_matrix)

Vocabulary: ['Another', 'This', 'a', 'but', 'containing', 'different', 'document', 'is', 'sample', 'similar', 'some', 'with', 'words', 'words.']
PPMI Matrix:
[[0.         0.         0.         0.49642982 0.         0.49642982
  0.11928842 0.         0.         0.         0.         0.49642982
  0.49642982 0.        ]
 [0.         0.         0.56654895 0.         0.13613753 0.
  0.13613753 0.56654895 0.56654895 0.         0.         0.
  0.         0.        ]
 [0.         0.55633203 0.         0.         0.13368248 0.
  0.13368248 0.55633203 0.55633203 0.         0.13368248 0.
  0.         0.13368248]
 [0.43722925 0.         0.         0.         0.10506296 0.43722925
  0.10506296 0.         0.         0.43722925 0.10506296 0.43722925
  0.43722925 0.10506296]
 [0.         0.21821789 0.21821789 0.21821789 0.         0.21821789
  0.43643578 0.21821789 0.21821789 0.21821789 0.43643578 0.21821789
  0.21821789 0.43643578]
 [0.43966253 0.         0.         0.43966253 0.10564766 0.
  0.10564